## NBIC Workflow Provena Toy Example

This relies on some pre-registered components and is intended to show an example provenance enabled workflow by integrating with the provena APIs. 

The actual computation/validation has been stripped from the source notebook - but hopefully it will be clear where this can be reintroduced

### Provena workflow configuration setup

In [13]:
# This is a small helper class which provides a config object for validation and
# a loader function
import workflow_config

# this contains helpers for interacting with the registry
import registry

# This is a helper function for managing authentication with Provena
import mdsisclienttools.auth.TokenManager as ProvenaAuth


import json
from pydantic import BaseModel
import time
import requests


In [12]:
# Provena config
stage = "DEV"
kc_endpoint = "https://auth.dev.rrap-is.com/auth/realms/rrap"
registry_endpoint = "https://registry-api.dev.rrap-is.com"
provenance_endpoint = "https://prov-api.dev.rrap-is.com"

In [3]:
# sets up auth connections - could potentially open browser window if not signed
# in recently - caches in .tokens.json - ensure this is included in gitignore
provena_auth = ProvenaAuth.DeviceFlowManager(
    stage=stage,
    keycloak_endpoint=kc_endpoint
)

# expose the get auth function which is used for provena methods 
get_auth = provena_auth.get_auth

No storage or object provided, using default location: .tokens.json.
Using storage type: FILE.
Using DEVICE auth flow.
Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Trying to use found tokens to refresh the access token.

Token refresh successful.



In [4]:
# Start by loading the config from the specified path 

# NOTE this could change from run to run - this holds all information required to run this model
config_path = "configs/example_workflow.json"
config = workflow_config.load_config(path=config_path)
config.pprint()


{
  "inputs": {
    "hourly_temperature": "10378.1/1709248",
    "hourly_temperature_template": "10378.1/1709254",
    "relative_humidity": "10378.1/1709249",
    "relative_humidity_template": "10378.1/1709255",
    "wind_speed": "10378.1/1709251",
    "wind_speed_template": "10378.1/1709256",
    "daily_mc_adf": "10378.1/1709252",
    "daily_mc_adf_template": "10378.1/1709257"
  },
  "outputs": {
    "hourly_ffdi": "10378.1/1709260",
    "hourly_ffdi_template": "10378.1/1709258"
  },
  "associations": {
    "person": "10378.1/1709236",
    "organisation": "10378.1/1709247"
  },
  "workflow_configuration": {
    "workflow_template": "10378.1/1709259"
  }
}


In [5]:
# let's validate the workflow config - this fetches ALL items referenced in the
# workflow json to ensure the items are valid 
valid = config.validate_entities(registry_endpoint=registry_endpoint, auth=get_auth)

if not valid:
    print("FAILED VALIDATION")
    raise Exception("Workflow config validation exception occurred. See output above.")


Validating registered Provena entities in config
Validating registered input datasets...
Fetching from registry, id: 10378.1/1709248...
Fetching from registry, id: 10378.1/1709249...
Fetching from registry, id: 10378.1/1709251...
Fetching from registry, id: 10378.1/1709252...
Fetching from registry, id: 10378.1/1709254...
Fetching from registry, id: 10378.1/1709255...
Fetching from registry, id: 10378.1/1709256...
Fetching from registry, id: 10378.1/1709257...
Validating registered output datasets...
Fetching from registry, id: 10378.1/1709260...
Fetching from registry, id: 10378.1/1709258...
Validating registered associations...
Fetching from registry, id: 10378.1/1709236...
Fetching from registry, id: 10378.1/1709247...
Validating registered associations...
Fetching from registry, id: 10378.1/1709259...
Validation successful...


## Model run integration
Now that the validation of the workflow configuration (incl. registered entitites) is complete - we can move into the example of running the model against this configuration.

### Data paths

Let's consider how data can be loaded. 

This example is straight from the notebook: 

```python
temperature = xr.open_zarr(nbic.catalog_s3_stage1.weather.projected.to_path('AU_hourly_temperature_C.zarr'), chunks='auto')
```

In this case we have a mechanism to reference the variable part of the data path, but hardcode the zarr file path. 

We could continue to operate in this manner, or we could utilise the external reposited path metadata in the registered dataset instead. I will demonstrate this below.

In [6]:
# let's establish the paths of the input from the dataset
def pprint_json(content) -> None:
    print(json.dumps(content,indent=2))

# fetch the dataset 
ds_id = config.inputs.hourly_temperature
fetched_ds = registry.fetch_dataset(registry_endpoint=registry_endpoint, id=ds_id, auth=get_auth())
pprint_json(fetched_ds)

Fetching from registry, id: 10378.1/1709248...
{
  "display_name": "NBIC Hourly Temperature",
  "collection_format": {
    "associations": {
      "organisation_id": "10378.1/1709247"
    },
    "dataset_info": {
      "name": "NBIC Hourly Temperature",
      "description": "Used in the example HourlyFFDI workflow. \n\nContains the hourly temperature. Hosted on external storage.",
      "access_info": {
        "reposited": false,
        "uri": "s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/projected/AU_hourly_temperature_C.zarr",
        "description": "This is hosted on the EASI platform in the NBIC stage one bucket. Contains a zarr file used for processing."
      },
      "publisher_id": "10378.1/1709247",
      "created_date": "2023-06-06",
      "published_date": "2023-06-06",
      "license": "https://gbrrestoration.github.io/rrap-mds-knowledge-hub/information-system/licenses.html#copyright-all-rights-reserved-",
      "preferred_citation": null,
      "keywords": [
   

In [7]:
# determine the external reposit path
file_path = fetched_ds["collection_format"]["dataset_info"]["access_info"]["uri"]
print(file_path)

s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/projected/AU_hourly_temperature_C.zarr


As demonstrated above, it is possible to retrieve the associated file path from the registered Dataset (assuming this info was included at registration time). Or the existing file path mechanism could continue being used.

A similar approach works for the other inputs. Shown below.

In [8]:
def fetch_and_path(id: str):
    dataset = registry.fetch_dataset(registry_endpoint=registry_endpoint, id=id, auth=get_auth())
    path = dataset["collection_format"]["dataset_info"]["access_info"]["uri"]
    return dataset, path

# temperature
temperature_ds, temperature_path = fetch_and_path(id=config.inputs.hourly_temperature)

# humidity
humidity_ds, humidity_path = fetch_and_path(id=config.inputs.relative_humidity)

# wind speed
wind_speed_ds, wind_speed_path = fetch_and_path(id=config.inputs.wind_speed)

# mc adf
mc_adf_ds, mc_adf_path = fetch_and_path(id=config.inputs.daily_mc_adf)

pprint_json({
   "Temperature: " : temperature_path,
   "Humidity: " : humidity_path,
   "Wind Speed: " : wind_speed_path,
   "Mc ADF: " : mc_adf_path,
})

Fetching from registry, id: 10378.1/1709248...
Fetching from registry, id: 10378.1/1709249...
Fetching from registry, id: 10378.1/1709251...
Fetching from registry, id: 10378.1/1709252...
{
  "Temperature: ": "s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/projected/AU_hourly_temperature_C.zarr",
  "Humidity: ": "s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/projected/AU_hourly_relative_humidity_percent.zarr",
  "Wind Speed: ": "s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/baseline/AU_hourly_wind_speed_mps_updated_220224_corrected.zarr",
  "Mc ADF: ": "s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/projected/AU_daily_McADF_improved_uncapped.zarr"
}


Alternatively, if we wanted to use the data storage utilities of the Provena data store, we could register a reposited dataset, and use the dynamic credential generation to produce r or r/w credentials into that specific dataset.

### Running our fake model

We are going to pretend to produce some output from this process.

In [9]:
def fake_data_fetch(path: str) -> int:
    # This method would take the path and return the data
    return 0

fake_temperature = fake_data_fetch(temperature_path)
fake_humidity = fake_data_fetch(humidity_path)
fake_wind_speed = fake_data_fetch(wind_speed_path)
fake_mc_adf = fake_data_fetch(mc_adf_path)

def fake_model(temperature: int, humidity: int, wind_speed : int, mc_adf: int) -> int:
    # this model does some heavy lifting and takes 10 seconds to finish 
    time.sleep(10) 
    
    return 0

# let's run our model with the inputs 

# start timer
start_time = int(time.time())

# run the model 
fake_model_output = fake_model(
    temperature=fake_temperature,
    humidity=fake_humidity,
    wind_speed=fake_wind_speed,
    mc_adf=fake_mc_adf
)

end_time = int(time.time())

print(f"Ran fake hourly FFDI calculation, took {end_time - start_time} seconds.")



Ran fake hourly FFDI calculation, took 10 seconds.


Now we have ran the toy model, let's register a provenance record which records the model run, the inputs used, and the outputs produced.

We need to think more about the output. 

There are two primary ways that Provena supports registering the results of a model run. 

1. Dynamically register a new Dataset and link to this dataset. This is the _preferred_ method as it creates a clear causal chain between the model and the output dataset.
2. Use a deferred or defined resource in an output dataset template to register the outputs into an existing dataset. E.g. overwrite an existing file or contribute new files to an existing dataset. This method produces less structurally clear provenance chains and may obfuscate the history of data (if overwriting).

We will show both methods. Method 2) is more closely aligned with the existing data path workflow in NBIC.

Model runs satisfy the following JSON schema

```json
{
  "workflow_template_id": "string",
  "inputs": [
    {
      "dataset_template_id": "string",
      "dataset_id": "string",
      "dataset_type": "DATA_STORE",
      "resources": {
        "additionalProp1": "string",
        "additionalProp2": "string",
        "additionalProp3": "string"
      }
    }
  ],
  "outputs": [
    {
      "dataset_template_id": "string",
      "dataset_id": "string",
      "dataset_type": "DATA_STORE",
      "resources": {
        "additionalProp1": "string",
        "additionalProp2": "string",
        "additionalProp3": "string"
      }
    }
  ],
  "annotations": {
    "additionalProp1": "string",
    "additionalProp2": "string",
    "additionalProp3": "string"
  },
  "description": "string",
  "associations": {
    "modeller_id": "string",
    "requesting_organisation_id": "string"
  },
  "start_time": 0,
  "end_time": 0
}
```

#### Overwrite an existing output at a specified path

Let's start with method 2) and overwrite a specified output. This dataset is pre-registered and is included in our config.

In [10]:
### Overwrite existing output

output_dataset_id = config.outputs.hourly_ffdi

# we can resolve the path using the same approach as above, or using existing
# NBIC path structure

output_ds, output_path =fetch_and_path(id=output_dataset_id)

pprint_json(output_ds)

Fetching from registry, id: 10378.1/1709260...
{
  "display_name": "NBIC Daily FFDI Preregistered Output",
  "collection_format": {
    "associations": {
      "organisation_id": "10378.1/1709247"
    },
    "dataset_info": {
      "name": "NBIC Daily FFDI Preregistered Output",
      "description": "Used in the example HourlyFFDI workflow. \n\nThis contains a reference to the preregistered output. Supports overwritten workflow style where the output of the model overwrites an existing resource.",
      "access_info": {
        "reposited": false,
        "uri": "s3://nbic1-stage-shared-artifacts/nbic-stage1/weather/projected/AU_hourly_FFDI.zarr",
        "description": "This is hosted on the EASI platform in the NBIC stage one bucket. Contains a zarr file which is the output of Hourly FFDI workflow."
      },
      "publisher_id": "10378.1/1709247",
      "created_date": "2023-06-06",
      "published_date": "2023-06-06",
      "license": "https://gbrrestoration.github.io/rrap-mds-kno

In [11]:
# building the model run payload 
model_run_payload = {
  # This specifies the workflow template we are using 
  "workflow_template_id": config.workflow_configuration.workflow_template,
  
  # here we specify a dataset to match each template, and optionally any
  # deferred resources
  "inputs": [
    # temperature
    {
      "dataset_template_id": config.inputs.hourly_temperature_template,
      "dataset_id": config.inputs.hourly_temperature,
      "dataset_type": "DATA_STORE",
      "resources": {
        "hourly_temperature_zarr": temperature_path,
      }
    },
    # humidity
    {
      "dataset_template_id": config.inputs.relative_humidity_template,
      "dataset_id": config.inputs.relative_humidity,
      "dataset_type": "DATA_STORE",
      "resources": {
        "relative_humidity_zarr": humidity_path,
      }
    },
    # wind speed
    {
      "dataset_template_id": config.inputs.wind_speed_template,
      "dataset_id": config.inputs.wind_speed,
      "dataset_type": "DATA_STORE",
      "resources": {
        "wind_speed_zarr": wind_speed_path
      }
    },
    # Mc ADF
    {
      "dataset_template_id": config.inputs.daily_mc_adf_template,
      "dataset_id": config.inputs.daily_mc_adf,
      "dataset_type": "DATA_STORE",
      "resources": {
        "mcadf_zarr": mc_adf_path
      }
    },
  ],
  "outputs": [
    # Hourly FFDI output
    {
      "dataset_template_id": config.outputs.hourly_ffdi_template,
      "dataset_id": config.outputs.hourly_ffdi,
      "dataset_type": "DATA_STORE",
      "resources": {
        "hourly_ffdi_zarr" : output_path
      }
    },
  ],
  "description": "HourlyFFDI Workflow. NBIC example. This is an example of using the FFDI workflow to register into an existing dataset.",
  "associations": {
    "modeller_id": config.associations.person,
    "requesting_organisation_id": config.associations.organisation
  },
  "start_time": start_time, 
  "end_time": end_time
}

In [14]:
# Registering the model run 
endpoint = provenance_endpoint + "/model_run/register_complete"
payload = model_run_payload

# send off request
print("Registering model run")
response = requests.post(url=endpoint, json=payload, auth=get_auth())

# use helper function to check response
registry.check_response(response=response, status_check=True)

response_content = response.json()
record_info = response_content["record_info"]
pprint_json(record_info)


Registering model run
Token validation failed due to error: Signature has expired.
Refreshing using refresh token

{
  "id": "10378.1/1709261",
  "prov_json": "{\"prefix\": {\"default\": \"http://hdl.handle.net/\"}, \"activity\": {\"10378.1/1709261\": {\"model_run/10378.1/1709261\": true, \"item_category\": \"ACTIVITY\", \"item_subtype\": \"MODEL_RUN\"}}, \"entity\": {\"10378.1/1709248\": {\"model_run/10378.1/1709261\": true, \"item_category\": \"ENTITY\", \"item_subtype\": \"DATASET\"}, \"10378.1/1709249\": {\"model_run/10378.1/1709261\": true, \"item_category\": \"ENTITY\", \"item_subtype\": \"DATASET\"}, \"10378.1/1709251\": {\"model_run/10378.1/1709261\": true, \"item_category\": \"ENTITY\", \"item_subtype\": \"DATASET\"}, \"10378.1/1709252\": {\"model_run/10378.1/1709261\": true, \"item_category\": \"ENTITY\", \"item_subtype\": \"DATASET\"}, \"10378.1/1709260\": {\"model_run/10378.1/1709261\": true, \"item_category\": \"ENTITY\", \"item_subtype\": \"DATASET\"}, \"10378.1/1709259\"